# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的远洋航运订舱销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.
[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？
[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。

2.
[客户问题] 你们的服务覆盖哪些港口？
[销售回答] 我们的服务网络遍及全球主要港口，从亚洲的上海、新加坡，到欧洲的鹿特丹、汉堡，以及美洲的洛杉矶、纽约等。无论您的货物需要运往何处，我们都能够提供无缝对接的物流解决方案。

3.
[客户问题] 运输过程中如何保证我的货物安全？
[销售回答] 安全是我们的首要任务。我们采用先进的货物追踪技术和严格的运输规程，确保您的货物全程监控。此外，我们还提供全面的保险选项，以减轻任何潜在的风险。

4.
[客户问题] 我们有一批货物急需尽快送达，你们能提供加急服务吗？
[销售回答] 当然可以。我们有专门的快速通道服务，能够优先安排您的货物装船，并通过优化路线规划来缩短运输时间。请告诉我具体的时间要求，我们将尽全力满足您的需求。

5.
[客户问题] 出口所需的文件有哪些？我该如何准备？
[销售回答] 出口文件通常包括商业发票、装箱单、原产地证书等。我们会提供详细的文档清单，并指导您完成所有必要的手续。我们的专业团队也可以协助处理复杂的文件工作，确保您的货物顺利通关。

6.
[客户问题] 如果我们建立长期合作关系，你们能提供什么额外的价值？
[销售回答] 长期合作伙伴对我们非常重要。我们可以提供更稳定的价格策略、优先服务和定制化解决方案。此外，我们还会定期进行业务回顾，以持续优化服务流程，确保您的满意度。

7.
[客户问题] 你们的业务如何符合环保标准？
[销售回答] 我们致力于可持续发展，遵守国际环保法规，如IMO的限硫令。我们投资于高效节能的船只，并优化航线减少碳排放。选择我们，意味着您也在支持绿色物流。

8.
[客户问题] 你们使用了哪些先进技术来提升服务质量？
[销售回答] 我们利用物联网（IoT）、人工智能（AI）和大数据分析来优化运营效率和客户服务。例如，我们的智能平台可以实时更新货物状态，让您随时掌握最新信息。

9.
[客户问题] 你们接受哪些支付方式？
[销售回答] 我们接受多种支付方式，包括电汇、信用证和第三方支付平台。我们也会根据您的财务状况和信用记录，提供灵活的账期安排。

10.
[客户问题] 我们需要特定的运输条件，你们能否提供定制化服务？
[销售回答] 绝对可以。我们有经验丰富的物流专家，可以根据您的具体需求设计个性化的运输方案。无论是温度控制还是特殊包装，我们都将全力以赴满足您的要求。


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [25]:
with open("real_estate_sales_data.txt", 'r', encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [26]:
from langchain.text_splitter import CharacterTextSplitter

In [27]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 150,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [28]:
docs = text_splitter.create_documents([real_estate_sales])

In [29]:
docs[0]

Document(page_content='[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？\n[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。')

In [30]:
len(docs)

80

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url="https://apikeyplus.com/v1"))

In [32]:
query = "现在仓位贵不贵？"

In [33]:
answer_list = db.similarity_search(query)

In [34]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我们需要了解旺季时的价格波动情况。
[销售回答] 在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。

[客户问题] 我们对价格时效性有要求，你们的报价有效期是多久？
[销售回答] 我们的报价一般有效期为7天，但在市场波动较大的时期，有效期可能会缩短。请尽快确认，以便我们为您保留仓位和价格。

[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？
[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。

[客户问题] 我们需要一个仓位报价单，包括所有附加费用。
[销售回答] 绝对没问题，我们将为您提供一份详细的报价单，包含基本运费、燃油附加费、港口处理费以及任何可能的旺季附加费。请确认您的电子邮件地址，我会立即发送给您。



In [35]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [36]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [37]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017B46722620>, search_kwargs={'k': 3})

In [38]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我们需要了解旺季时的价格波动情况。
[销售回答] 在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。

[客户问题] 我们对价格时效性有要求，你们的报价有效期是多久？
[销售回答] 我们的报价一般有效期为7天，但在市场波动较大的时期，有效期可能会缩短。请尽快确认，以便我们为您保留仓位和价格。

[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？
[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。



In [47]:
docs = topK_retriever.get_relevant_documents("你们有没有价格1w块钱的仓吗？")

In [48]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我们对价格时效性有要求，你们的报价有效期是多久？
[销售回答] 我们的报价一般有效期为7天，但在市场波动较大的时期，有效期可能会缩短。请尽快确认，以便我们为您保留仓位和价格。

[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？
[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。

[客户问题] 我们想知道如果预订量大，你们是否有折扣？
[销售回答] 对于大宗预订，我们确实提供折扣。请告诉我们您的具体需求和预订量，我们将为您计算相应的折扣幅度。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [54]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7}
)

In [55]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我们需要了解旺季时的价格波动情况。
[销售回答] 在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。

[客户问题] 我们对价格时效性有要求，你们的报价有效期是多久？
[销售回答] 我们的报价一般有效期为7天，但在市场波动较大的时期，有效期可能会缩短。请尽快确认，以便我们为您保留仓位和价格。

[客户问题] 我们在考虑成本，你们的价格是否可以再优惠一些？
[销售回答] 我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。

[客户问题] 我们需要一个仓位报价单，包括所有附加费用。
[销售回答] 绝对没问题，我们将为您提供一份详细的报价单，包含基本运费、燃油附加费、港口处理费以及任何可能的旺季附加费。请确认您的电子邮件地址，我会立即发送给您。



### 提取向量数据库中的`销售回答`

In [56]:
docs = retriever.get_relevant_documents(query)

In [57]:
docs[0].page_content

'[客户问题] 我们需要了解旺季时的价格波动情况。\n[销售回答] 在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。'

In [58]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我们需要了解旺季时的价格波动情况。\n',
 '在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。']

In [59]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [60]:
ans

'在旺季，价格可能会因需求增加而上涨。我们会提前通知您任何预计的价格变动，并尽可能提供锁定价格的选项，以避免不必要的成本增加。'

#### 尝试各种问题

In [61]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [65]:
query = "我想去医院"

print(sales(query))

[]


D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [66]:
print(sales(query, 0.75))

[]


D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


In [67]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['我完全理解您对成本控制的关注。我们的服务定价是基于市场最优标准，同时我们提供灵活的合同条款来适应您的业务需求。如果您能告诉我您的具体运输量和频率，我或许可以为您提供一个更加个性化的报价方案。', '我们理解成本控制的重要性，我们会根据您的具体需求，结合市场状况，提供最具竞争力的价格。此外，我们还会定期审查成本结构，确保您始终获得最佳价值。', '我们承诺价格透明，所有费用都会在报价单上明确列出，包括可能的额外费用如码头费、报关费等。我们不会收取任何未事先告知的隐藏费用。', '绝对没问题，我们将为您提供一份详细的报价单，包含基本运费、燃油附加费、港口处理费以及任何可能的旺季附加费。请确认您的电子邮件地址，我会立即发送给您。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [68]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5, base_url="https://apikeyplus.com/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [69]:
qa_chain({"query": "你们小区有200万的房子吗？"})

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？',
 'result': "I don't have access to real estate listings or current housing market information, so I cannot provide details about house prices in specific communities. Please consult a real estate website or contact a local real estate agent for information about house prices in a particular area."}

In [70]:
qa_chain({"query": "小区吵不吵"})

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '小区吵不吵',
 'result': '"小区吵不吵"这个问题的答案取决于具体的小区环境和个人的感受。不同的小区因为地理位置、建筑布局、居民构成以及附近的交通和商业设施等因素，其噪音水平会有很大差异。有些小区可能会比较安静，而有些则可能因为邻近繁忙道路或有娱乐设施而较为吵闹。\n\n如果你是想了解某个具体小区的噪音状况，建议你可以在该小区周边走走，特别是在晚上或者早晨高峰时段，亲自感受一下噪音水平。同时，你也可以询问该小区的居民或物业管理人员，或者上网查找相关的居住评价和论坛讨论，以获得更多信息。'}

In [71]:
print(sales("小区吵不吵"))

[]


D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [72]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url="https://apikeyplus.com/v1"), allow_dangerous_deserialization=True)

In [73]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5, base_url="https://apikeyplus.com/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [74]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么', 'result': '对不起，我无法帮助您购买别墅。我是一个人工智能，我的主要功能是提供信息和回答问题。'}

In [75]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [76]:
qa_chain({"query": "我想买别墅，你们有么"})

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我无法帮助您购买别墅，因为我是一个人工智能，主要用于提供信息和回答问题。'}

In [77]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [78]:
result = qa_chain({"query": "我想买别墅，你们有么"})

D:\Software\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


In [79]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法帮助您购买别墅，因为我是一个人工智能，不提供房地产服务。',
 'source_documents': []}